In [1]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [2]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df


class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        
        #--------------------------------
        expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        expr_var = [pl.var(col).alias(f"var_{col}") for col in cols]

#         expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
#         expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]
        #--------------------------------
        
        
        return expr_max +expr_last+expr_mean+expr_var+expr_count#+expr_min+expr_median
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        
        #--------------------------------
#         expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
#         expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        #--------------------------------
        
        
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]

        return  expr_max +expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        
        #--------------------------------
        expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        #--------------------------------
        
        return  expr_max +expr_last+expr_count
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

In [4]:
%%time
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_other_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_person_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
    ]
}

CPU times: user 5min 13s, sys: 1min 40s, total: 6min 54s
Wall time: 2min 21s


In [5]:
%%time
df_train = feature_eng(**data_store)
print("train data shape:\t", df_train.shape)
del data_store
gc.collect()
df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
print("train data shape:\t", df_train.shape)
nums=df_train.select_dtypes(exclude='category').columns
from itertools import combinations, permutations
#df_train=df_train[nums]
nans_df = df_train[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
            #print(str(gg)+'-'+str(n),', ',end='')
        use.append(vx)
        #print()
    print('Use these',use)
    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    # 计算列之间的相关性
    correlation_matrix = matrix.corr()

    # 分组列
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
    return groups

uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+list(df_train.select_dtypes(include='category').columns)
print(len(uses))
df_train=df_train[uses]

train data shape:	 (1526659, 893)
Memory usage of dataframe is 5011.39 MB
Memory usage after optimization is: 1736.99 MB
Decreased by 65.3%
train data shape:	 (1526659, 506)
Use these ['case_id', 'WEEK_NUM', 'target', 'month_decision', 'weekday_decision', 'credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'isbidproduct_1095L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L']
####### NAN count = 0
####### NAN count = 918

In [6]:
df_train.head(10)

,case_id,WEEK_NUM,target,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,max_cacccardblochreas_147M,last_cacccardblochreas_147M,max_conts_type_509L,last_conts_type_509L,max_conts_role_79M,max_empls_economicalst_849M,max_empls_employer_name_740M,last_conts_role_79M,last_empls_economicalst_849M,last_empls_employer_name_740M
0,0,0,0,1,4,30000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,1,4,19999.800781,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0,1,5,78000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,PRIMARY_MOBILE,EMPLOYMENT_PHONE,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0,1,4,40000.000000,0.0,1.0,0.0,2.0,...,NaN,NaN,PRIMARY_MOBILE,PRIMARY_EMAIL,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,1,1,5,44000.000000,0.0,1.0,0.0,0.0,...,NaN,NaN,PRIMARY_MOBILE,HOME_PHONE,NaN,NaN,NaN,NaN,NaN,NaN
5,5,0,0,1,3,60000.000000,0.0,1.0,0.0,8.0,...,NaN,NaN,PRIMARY_MOBILE,PRIMARY_MOBILE,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1
6,6,0,0,1,4,20000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,PRIMARY_MOBILE,EMPLOYMENT_PHONE,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1
7,7,0,0,1,4,20300.000000,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1
8,8,0,0,1,4,40000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1
9,9,0,0,1,4,64000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1


In [7]:
sample = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv")
device='cpu'
#n_samples=200000
n_est=600
# DRY_RUN = True if sample.shape[0] == 10 else False   
# if DRY_RUN:
#     device='cpu'
#     df_train = df_train.iloc[:50000]
#     #n_samples=10000
#     n_est=600
print(device)

cpu


In [8]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
#         read_file(TEST_DIR / "test_other_1.parquet", 1),
#         read_file(TEST_DIR / "test_person_1.parquet", 1),
#         read_file(TEST_DIR / "test_deposit_1.parquet", 1),
#         read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_file(TEST_DIR / "test_applprev_2.parquet", 2),
        read_file(TEST_DIR / "test_person_2.parquet", 2)
    ]
}

In [9]:
#(set([col for col in df_train.columns if col != "target"])).difference(set(df_test.columns))

In [10]:
df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)
del data_store
gc.collect()
df_test = df_test.select([col for col in df_train.columns if col != "target"])
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

gc.collect()

test data shape:	 (10, 892)
train data shape:	 (1526659, 389)
test data shape:	 (10, 388)
Memory usage of dataframe is 0.03 MB
Memory usage after optimization is: 0.02 MB
Decreased by 46.0%


0

### Feature Selection

In [11]:
y = df_train["target"]
weeks = df_train["WEEK_NUM"]
df_train= df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)


In [12]:
df_train[cat_cols] = df_train[cat_cols].astype(str)
df_test[cat_cols] = df_test[cat_cols].astype(str)

In [13]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,  
    "learning_rate": 0.05,
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "device": device, 
    "verbose": -1,
}

params2 = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_depth": 10,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "alpha": 0.1,  
    "lambda": 10,  
    "tree_method": 'gpu_hist' if device == 'gpu' else 'auto',
    "random_state": 42,
    "verbosity": 0,
    "enable_categorical":True,
}

In [14]:
%%time
from catboost import CatBoostClassifier, Pool
import xgboost as xgb

fitted_models_cat = []
fitted_models_lgb = []
fitted_models_xgb = []

cv_scores_cat = []
cv_scores_lgb = []
cv_scores_xgb = []


for idx_train, idx_valid in cv.split(df_train, y, groups=weeks):#
    X_train, y_train = df_train.iloc[idx_train], y.iloc[idx_train]# 
    X_valid, y_valid = df_train.iloc[idx_valid], y.iloc[idx_valid]
    train_pool = Pool(X_train, y_train,cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid,cat_features=cat_cols)
    clf = CatBoostClassifier(
    eval_metric='AUC',
    task_type='GPU',
    learning_rate=0.03,
    iterations=n_est)
    random_seed=3107
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    fitted_models_cat.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_cat.append(auc_score)
    
    
    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")
    
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)] )
    
    fitted_models_lgb.append(model)
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_lgb.append(auc_score)
    
    
    model2 = xgb.XGBClassifier(**params2)
    model2.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=100, verbose=False)
    
    fitted_models_xgb.append(model2)
    
    y_pred_valid = model2.predict_proba(X_valid)[:, 1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_xgb.append(auc_score)
    
    del clf, model, model2
    gc.collect()
    
    
print("CV AUC scores: ", cv_scores_cat)
print("Maximum CV AUC score: ", max(cv_scores_cat))


print("CV AUC scores: ", cv_scores_lgb)
print("Maximum CV AUC score: ", max(cv_scores_lgb))

print("CV AUC scores: ", cv_scores_xgb)
print("Maximum CV AUC score: ", max(cv_scores_xgb))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6702003	best: 0.6702003 (0)	total: 772ms	remaining: 7m 42s
300:	test: 0.8234128	best: 0.8234128 (300)	total: 2m 44s	remaining: 2m 43s
599:	test: 0.8294433	best: 0.8294433 (599)	total: 5m 21s	remaining: 0us
bestTest = 0.8294433057
bestIteration = 599
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.838886
[400]	valid_0's auc: 0.843324
[600]	valid_0's auc: 0.844853
[800]	valid_0's auc: 0.845327
[1000]	valid_0's auc: 0.845663
Early stopping, best iteration is:
[1056]	valid_0's auc: 0.845751


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7024407	best: 0.7024407 (0)	total: 582ms	remaining: 5m 48s
300:	test: 0.8246672	best: 0.8246672 (300)	total: 2m 44s	remaining: 2m 43s
599:	test: 0.8315626	best: 0.8315626 (599)	total: 5m 21s	remaining: 0us
bestTest = 0.8315625787
bestIteration = 599
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.840796
[400]	valid_0's auc: 0.844893
[600]	valid_0's auc: 0.846529
[800]	valid_0's auc: 0.846998
[1000]	valid_0's auc: 0.847297
Early stopping, best iteration is:
[1021]	valid_0's auc: 0.847355


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6928050	best: 0.6928050 (0)	total: 604ms	remaining: 6m 1s
300:	test: 0.8292674	best: 0.8292674 (300)	total: 2m 44s	remaining: 2m 43s
599:	test: 0.8353859	best: 0.8353859 (599)	total: 5m 21s	remaining: 0us
bestTest = 0.8353859186
bestIteration = 599
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.844758
[400]	valid_0's auc: 0.849158
[600]	valid_0's auc: 0.850625
[800]	valid_0's auc: 0.851262
[1000]	valid_0's auc: 0.85176
[1200]	valid_0's auc: 0.851883
Early stopping, best iteration is:
[1141]	valid_0's auc: 0.851922


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6878951	best: 0.6878951 (0)	total: 573ms	remaining: 5m 43s
300:	test: 0.8276699	best: 0.8276699 (300)	total: 2m 44s	remaining: 2m 43s
599:	test: 0.8339594	best: 0.8339594 (599)	total: 5m 22s	remaining: 0us
bestTest = 0.8339594007
bestIteration = 599
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.843769
[400]	valid_0's auc: 0.847987
[600]	valid_0's auc: 0.84925
[800]	valid_0's auc: 0.849958
[1000]	valid_0's auc: 0.85046
[1200]	valid_0's auc: 0.850701
[1400]	valid_0's auc: 0.850867
Early stopping, best iteration is:
[1336]	valid_0's auc: 0.850933


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6958654	best: 0.6958654 (0)	total: 608ms	remaining: 6m 4s
300:	test: 0.8233147	best: 0.8233147 (300)	total: 2m 46s	remaining: 2m 45s
599:	test: 0.8301074	best: 0.8301074 (599)	total: 5m 25s	remaining: 0us
bestTest = 0.8301074207
bestIteration = 599
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.840135
[400]	valid_0's auc: 0.844578
[600]	valid_0's auc: 0.845889
[800]	valid_0's auc: 0.846231
[1000]	valid_0's auc: 0.846554
Early stopping, best iteration is:
[1081]	valid_0's auc: 0.846685
CV AUC scores:  [0.8294432528052653, 0.8315626066120899, 0.8353859238250214, 0.8339593853784014, 0.8301074155437911]
Maximum CV AUC score:  0.8353859238250214
CV AUC scores:  [0.8457512429215103, 0.8473552243290552, 0.8519223286016897, 0.8509331792798736, 0.8466846220154489]
Maximum CV AUC score:  0.8519223286016897
CV AUC scores:  [0.8373816401484828, 0.8361844697468656, 0.8439355099577903, 0.8413175998561633, 0.8365180374232661]
Maximum CV AUC score:  0.8

In [15]:
del y 
del weeks
del df_train

gc.collect()

0

In [16]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:10]]
        y_preds+=y_preds #tang trong so
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[10:]]
        print(len(y_preds))
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_cat+fitted_models_lgb+fitted_models_xgb)

In [17]:
del fitted_models_cat, fitted_models_lgb, fitted_models_xgb
gc.collect()

0

# Submision

In [18]:
df_test = df_test.drop(columns=["WEEK_NUM"])
df_test = df_test.set_index("case_id")

In [19]:
y_pred = pd.Series(model.predict_proba(df_test)[:, 1], index=df_test.index)

df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred
df_subm.to_csv("submission.csv")
df_subm

25


,score
case_id,
57543,0.015784
57549,0.050048
57551,0.008468
57552,0.021078
57569,0.106309
57630,0.019734
57631,0.023241
57632,0.008056
57633,0.025834
